# Text Analysis - HW2
Samir Epili - se7982 - 8:30-10:30 section

Christian Lee - cnl878 - 10:30-12:30 section

Dylan Nikol - drn497 - 10:30-12:30 section

Luke Stevens - ls44426 - 8:30-10:30 section

Matthew Streichler - mrs4732 - 10:30-12:30 section

# Task A
Extract 5-6K reviews of 200-250 products

In [1]:
!pip install selenium
!apt-get -q update # to update ubuntu to correctly run apt install
!apt install -yq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

     |████████████████████████████████| 904 kB 62 kB/s eta 0:00:011
/bin/sh: apt-get: command not found
Unable to locate an executable at "/Library/Java/JavaVirtualMachines/jdk1.8.0_261.jdk/Contents/Home/bin/apt" (-1)
cp: /usr/lib/chromium-browser/chromedriver: No such file or directory


In [2]:
def altElement(l):
    return l[::2]

# removes extra data that comes with each comment
def split_comment(s):
    splitted = s.split('\n')
    comment_body = splitted[5:-2]
    return comment_body

# converts comments to a string and removes brackets
def list_to_string(x):
    string = str(x)
    return string.rstrip(']').lstrip('[')

In [3]:
import pandas as pd
driver = webdriver.Chrome('chromedriver', options=chrome_options)
driver.get('https://www.beeradvocate.com/beer/top-rated/')
urls = []
beer_names = []

# Get list of beer urls
url_paths = driver.find_elements_by_xpath('//*[contains(@href, "/beer/profile/")]')
for i in url_paths:
    urls.append(i.get_attribute('href'))
beer_urls = altElement(urls)

# Get list of beer names
beer_name = driver.find_elements_by_xpath('//*[contains(@href, "/beer/profile/")]')
for i in beer_name:
    beer_names.append(i.text)
beer_names = altElement(beer_names)

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
# scrape comments for each beer
beer = pd.DataFrame(columns = ['product_name','product_review','user_rating'])


for beer_page in beer_urls:
    driver.get(beer_page)
    ids = driver.find_elements_by_xpath("//*[contains(@id,'rating_fullview_container')]")
    comment_ids = []
    for i in ids:
        comment_ids.append(i.get_attribute('ba-user'))

    for x in comment_ids:
        
        product_name = driver.find_elements_by_xpath('//*[@id="content"]/div/div/div[3]/div/div/div[1]/h1')[0]
        product = product_name.text.split('\n')[0]

        user_message = driver.find_elements_by_xpath('//*[@ba-user= ' + x + ']')[0]
        comment = user_message.text

        user_rating = driver.find_elements_by_xpath('//*[@ba-user= ' + x + ']//*[@id="rating_fullview_content_2"]/span[2]')[0]
        rating = user_rating.text

        beer.loc[len(beer)] = [product,comment,rating]

beer['product_review'] = beer['product_review'].map(split_comment)
beer['product_review'] = beer['product_review'].map(list_to_string)

beer.to_csv('beer.csv')
print("Beer data exported to beer.csv :)")

In [8]:
import pandas as pd
import numpy as np
import nltk


df = pd.read_csv('beer.csv')

In [9]:
df = df.drop(['Unnamed: 0'], axis=1)
df.head()

,product_name,product_review,user_rating
0,Kentucky Brunch Brand Stout,'Smell: early morning pancakes and coffee befo...,5.00
1,Kentucky Brunch Brand Stout,"""2019 vintage. Pours a very dark brown color w...",4.53
2,Kentucky Brunch Brand Stout,"""It's hyped... There is a lot of breweries doi...",1.49
3,Kentucky Brunch Brand Stout,"""Reviewing 2019 vintage. This pours thick and ...",4.52
4,Kentucky Brunch Brand Stout,"""2018 version. Poured dark with a small head. ...",4.99


In [10]:
reviews = df.product_review
reviews.head()

0    'Smell: early morning pancakes and coffee befo...
1    "2019 vintage. Pours a very dark brown color w...
2    "It's hyped... There is a lot of breweries doi...
3    "Reviewing 2019 vintage. This pours thick and ...
4    "2018 version. Poured dark with a small head. ...
Name: product_review, dtype: object

# Task B
We assume that the customer has chosen the following attributes to describe a beer - complex, spicy, aroma

In [11]:
original_atts = ['complex', 'spicy', 'aroma']

# Task C
Perform cosine similarity analysis with three attributes - output product_name, product_review, similarity_score

In [12]:
#!pip install gensim
#!pip install translate
#!pip install spacy
#!python -m spacy download en_core_web_lg

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.parsing.preprocessing import remove_stopwords

import string
from string import punctuation
import translate

In [14]:
def stop_words_remover(review):
    review = remove_stopwords(review)
    return review

def combine(review):
    return "".join(review)

def cos_sim(review):
    orig_atts = " ".join(original_atts)
    review_att = [orig_atts, review]
    tfidf_vec = TfidfVectorizer()
    tfidf = tfidf_vec.fit_transform(review_att)
    return cosine_similarity(tfidf[0:1], tfidf)[0][1]

In [15]:
cos_df = pd.DataFrame(data=df[["product_name", "product_review", "user_rating"]])

cos_df["new_comment"] = df['product_review'].apply(lambda x : str(x).translate(str.maketrans('', '', string.punctuation)))
cos_df['new_comment'] = cos_df["new_comment"].map(stop_words_remover)
cos_df['new_comment'] = cos_df["new_comment"].map(combine)
cos_df['cos_sim'] = cos_df["new_comment"].map(cos_sim)

In [16]:
task_c_df = pd.DataFrame(data=cos_df[["product_name", "new_comment", "cos_sim"]])
task_c_df.rename(columns={"new_comment":"product_review", "cos_sim":"similarity_score"}, inplace=True)
task_c_df

,product_name,product_review,similarity_score
0,Kentucky Brunch Brand Stout,Smell early morning pancakes coffee work wakes...,0.000000
1,Kentucky Brunch Brand Stout,2019 vintage Pours dark brown color noticeable...,0.000000
2,Kentucky Brunch Brand Stout,Its hyped There lot breweries style beer right...,0.000000
3,Kentucky Brunch Brand Stout,Reviewing 2019 vintage This pours close black ...,0.034786
4,Kentucky Brunch Brand Stout,2018 version Poured dark small head Smell mapl...,0.000000
...,...,...,...
6225,Flora - Blueberry,375ml bottle Thanks Dustin Poured taster glass...,0.000000
6226,Flora - Blueberry,Reviewed notes 72714 A Pours deep glowing purp...,0.025810
6227,Flora - Blueberry,500 mL bottle poured snifter Appearance Nice p...,0.000000
6228,Flora - Blueberry,Poured 500 mL bottle snifter Reviewed notes ni...,0.027270


# Task D
Perform sentiment analysis for each attribute - output sentinment score for individual attributes and average sentiment score of all three attributes

In [17]:
# function version of above to find all variations of three selected attributes
import re

def update_list(attribute_list):
    match_list = []
    for i in reviews:
        for att in attribute_list:
            match = re.findall(att + '[?!.,]+', str(i))
            if match not in match_list and match != []:
                match_list.append(match)
                for x in match_list:
                    for y in x:
                        attribute_list.append(y)
    return set(attribute_list)

In [18]:
#install if necessary
#!pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [20]:
lex = SentimentIntensityAnalyzer(lexicon_file="vader_lexicon_beer.txt", \
                                 emoji_lexicon="emoji_utf8_lexicon.txt")

#lexicon used is different from the default lexicon
## changed hell -> 0.0, dangerously -> 0.0

In [21]:
def sentiment_analyzer_scores(sentence):
    score = lex.polarity_scores(sentence)
    #print("{:-<40} {}".format(sentence, str(score)))
    #return compound score only to add to DF
    comp = score['compound']
    return comp

def att_to_dict(attribute_list):
    att_dict = dict.fromkeys(attribute_list, 0)
    return att_dict

def add_to_dict(att, original_atts, att_dict):
    if att in original_atts:
        att_dict[att] += 1
    return att_dict        

In [22]:
# call list function to find all attributes with punctuation
att = update_list(original_atts)

In [23]:
# call list function to find all attributes with punctuation
original_atts = ['complex', 'spicy', 'aroma']

score_list = []

att_scores_df = pd.DataFrame(columns = original_atts)


for phrase in reviews:
    #create dictionary from original attribute list to keep track of composite scores for each attribute per review
    att_scores = att_to_dict(original_atts)
    
    score = 0
    count = 0
    att1_count = 0
    att2_count = 0
    att3_count = 0
    arr = str(phrase).replace("'",'').split()
    its = [iter(arr), iter(arr[1:]), iter(arr[2:]), iter(arr[3:]), iter(arr[4:])]
    parse = list(zip(its[0], its[1], its[2], its[3], its[4]))
    for i in range(len(parse)):
        if i == 0:
            if parse[i][0] in att:
                #print(parse[i][0])
                #print(sentiment_analyzer_scores(' '.join(parse[i])))
                score += sentiment_analyzer_scores(' '.join(parse[i]))
                count += 1        
                if original_atts[0] in parse[i][0]:
                    att_scores[original_atts[0]] += sentiment_analyzer_scores(' '.join(parse[i]))
                    att1_count += 1
                if original_atts[1] in parse[i][0]:
                    att_scores[original_atts[1]] += sentiment_analyzer_scores(' '.join(parse[i]))
                    att2_count += 1
                if original_atts[2] in parse[i][0]:
                    att_scores[original_atts[2]] += sentiment_analyzer_scores(' '.join(parse[i]))
                    att3_count += 1                                    
            if parse[i][1] in att:
                #print(parse[i][1])
                #print(sentiment_analyzer_scores(' '.join(parse[i])))
                score += sentiment_analyzer_scores(' '.join(parse[i]))
                count += 1
                if original_atts[0] in parse[i][1]:
                    att_scores[original_atts[0]] += sentiment_analyzer_scores(' '.join(parse[i]))
                    att1_count += 1
                if original_atts[1] in parse[i][1]:
                    att_scores[original_atts[1]] += sentiment_analyzer_scores(' '.join(parse[i]))
                    att2_count += 1
                if original_atts[2] in parse[i][1]:
                    att_scores[original_atts[2]] += sentiment_analyzer_scores(' '.join(parse[i]))
                    att3_count += 1
                    
        if i == len(parse)-1:
            if parse[i][3] in att:
                #print(parse[i][3])
                #print(sentiment_analyzer_scores(' '.join(parse[i])))
                score += sentiment_analyzer_scores(' '.join(parse[i]))
                count += 1
                if original_atts[0] in parse[i][3]:
                    att_scores[original_atts[0]] += sentiment_analyzer_scores(' '.join(parse[i]))
                    att1_count += 1
                if original_atts[1] in parse[i][3]:
                    att_scores[original_atts[1]] += sentiment_analyzer_scores(' '.join(parse[i]))
                    att2_count += 1
                if original_atts[2] in parse[i][3]:
                    att_scores[original_atts[2]] += sentiment_analyzer_scores(' '.join(parse[i]))
                    att3_count += 1
            if parse[i][4] in att:
                #print(parse[i][4])
                #print(sentiment_analyzer_scores(' '.join(parse[i])))
                score += sentiment_analyzer_scores(' '.join(parse[i]))
                count += 1
                if original_atts[0] in parse[i][4]:
                    att_scores[original_atts[0]] += sentiment_analyzer_scores(' '.join(parse[i]))
                    att1_count += 1
                if original_atts[1] in parse[i][4]:
                    att_scores[original_atts[1]] += sentiment_analyzer_scores(' '.join(parse[i]))
                    att2_count += 1
                if original_atts[2] in parse[i][4]:
                    att_scores[original_atts[2]] += sentiment_analyzer_scores(' '.join(parse[i]))
                    att3_count += 1
        if parse[i][2] in att:
            #print(parse[i][2])
            #print(sentiment_analyzer_scores(' '.join(parse[i])))
            score += sentiment_analyzer_scores(' '.join(parse[i]))
            count += 1
            if original_atts[0] in parse[i][2]:
                att_scores[original_atts[0]] += sentiment_analyzer_scores(' '.join(parse[i]))
                att1_count += 1
            if original_atts[1] in parse[i][2]:
                att_scores[original_atts[1]] += sentiment_analyzer_scores(' '.join(parse[i]))
                att2_count += 1
            if original_atts[2] in parse[i][2]:
                att_scores[original_atts[2]] += sentiment_analyzer_scores(' '.join(parse[i]))
                att3_count += 1
    
    if att1_count == 0:
        att_scores[original_atts[0]] = np.nan
    else:
        att_scores[original_atts[0]] = att_scores[original_atts[0]]/att1_count
    if att2_count == 0:
        att_scores[original_atts[1]] = np.nan
    else:
        att_scores[original_atts[1]] = att_scores[original_atts[1]]/att2_count
    if att3_count == 0:
        att_scores[original_atts[2]] = np.nan
    else:
        att_scores[original_atts[2]] = att_scores[original_atts[2]]/att3_count
        
    att_scores_df = att_scores_df.append(att_scores, ignore_index = True)
    
    if count == 0:
        score_list.append(np.nan)
        
    else:
        score_list.append(score/count)

In [24]:
# turn overall_sent_score into Series to append to beer DF
df['overall_sent_score'] = pd.Series(score_list)

# merge df with att_scores_df
beer_sent = df.merge(att_scores_df, left_index = True, right_index = True)

In [25]:
beer_sent.groupby(['product_name']).mean().sort_values(by = 'overall_sent_score', ascending = False)

,user_rating,overall_sent_score,complex,spicy,aroma
product_name,,,,,
Headroom,4.5168,0.750100,NaN,NaN,0.750100
Pirate Bomb!,4.4764,0.649175,0.59450,0.641150,0.704700
Pseudo Sue - Double Dry-Hopped,4.2992,0.546575,0.42150,NaN,0.588267
Black Tuesday - Reserve,4.5880,0.499825,0.65605,NaN,0.000000
Fou' Foune,4.5068,0.496250,0.59940,NaN,0.475620
...,...,...,...,...,...
Cellarman Barrel Aged Saison,4.4576,-0.034233,-0.05135,NaN,0.000000
Atrial Rubicite,4.5632,-0.051350,NaN,NaN,-0.051350
Fuzzy Baby Ducks,4.3100,-0.084300,0.00000,0.000000,-0.140500


# Task E
Assume an evaluation score for each beer

In [26]:
import sklearn
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()

agg_all_recs = task_c_df.merge(beer_sent, left_index = True, right_index = True)
all_recs_by_beer = agg_all_recs.groupby(['product_name_x']).mean()

scale = sklearn.preprocessing.scale
all_recs_by_beer['Recommendation_Score'] = scale(all_recs_by_beer['similarity_score']) + scale(all_recs_by_beer['overall_sent_score'])

result = all_recs_by_beer.sort_values(by='Recommendation_Score', ascending=False)
result

,similarity_score,user_rating,overall_sent_score,complex,spicy,aroma,Recommendation_Score
product_name_x,,,,,,,
Zenne Y Frontera,0.031206,4.7068,0.388415,0.392857,NaN,0.35680,4.297057
Leche Borracho,0.038135,4.4756,0.179663,0.130750,0.19765,0.15540,3.590068
Fou' Foune,0.020253,4.5068,0.496250,0.599400,NaN,0.47562,3.556548
Oude Geuze Cuvée Armand & Gaston,0.036483,4.4664,0.177522,0.079017,NaN,0.30584,3.324531
Pirate Bomb!,0.010105,4.4764,0.649175,0.594500,0.64115,0.70470,3.313941
...,...,...,...,...,...,...,...
Assassin,0.005486,4.5012,0.000000,0.000000,NaN,0.00000,-2.808800
Marshmallow Handjee,0.003103,4.7016,0.000000,NaN,NaN,0.00000,-3.166146
Heavy Mettle,0.009905,4.3500,-0.142600,NaN,NaN,-0.14260,-3.338885


We scaled similarity_score and overall_sent_score because a beer might have a low similarity and a high sentiment on 1 attribute which would output a higher recommendation score than what it should actually have.

In [27]:
result[0:3]

,similarity_score,user_rating,overall_sent_score,complex,spicy,aroma,Recommendation_Score
product_name_x,,,,,,,
Zenne Y Frontera,0.031206,4.7068,0.388415,0.392857,NaN,0.35680,4.297057
Leche Borracho,0.038135,4.4756,0.179663,0.130750,0.19765,0.15540,3.590068
Fou' Foune,0.020253,4.5068,0.496250,0.599400,NaN,0.47562,3.556548


We recommend that the a customer try the following three beers if they searched for complex, spicy, and aroma:
    1. Zenne Y Frontera
    2. Leche Borracho
    3. Fou' Foune

# Task F

In [28]:
import spacy 
import en_core_web_lg
nlp = en_core_web_lg.load()

def spacy_sim(review):
    r = nlp(review)
    orig_atts = " ".join(original_atts)
    sp_sim = nlp(orig_atts)
    return r.similarity(sp_sim)

In [29]:
task_c_df['spacy_similarity'] = task_c_df['product_review'].apply(spacy_sim)
task_c_df

,product_name,product_review,similarity_score,spacy_similarity
0,Kentucky Brunch Brand Stout,Smell early morning pancakes coffee work wakes...,0.000000,0.660525
1,Kentucky Brunch Brand Stout,2019 vintage Pours dark brown color noticeable...,0.000000,0.721600
2,Kentucky Brunch Brand Stout,Its hyped There lot breweries style beer right...,0.000000,0.476465
3,Kentucky Brunch Brand Stout,Reviewing 2019 vintage This pours close black ...,0.034786,0.616807
4,Kentucky Brunch Brand Stout,2018 version Poured dark small head Smell mapl...,0.000000,0.682492
...,...,...,...,...
6225,Flora - Blueberry,375ml bottle Thanks Dustin Poured taster glass...,0.000000,0.664760
6226,Flora - Blueberry,Reviewed notes 72714 A Pours deep glowing purp...,0.025810,0.778817
6227,Flora - Blueberry,500 mL bottle poured snifter Appearance Nice p...,0.000000,0.748480
6228,Flora - Blueberry,Poured 500 mL bottle snifter Reviewed notes ni...,0.027270,0.785140


In [30]:
spacy_by_beer = task_c_df.groupby(['product_name']).mean()['spacy_similarity']
spacy_by_beer

product_name
3rd Anniversary Imperial IPA               0.688106
4th Anniversary                            0.681356
A Deal With The Devil                      0.692171
A Deal With The Devil - Double Oak-Aged    0.662109
Aaron                                      0.704470
                                             ...   
Westly                                     0.677065
Wide Awake It's Morning                    0.706549
Zenne Y Frontera                           0.711061
Zombie Dust                                0.701286
§ucaba                                     0.710032
Name: spacy_similarity, Length: 250, dtype: float64

In [31]:
# add spacy_similarity to df grouped by beer
all_recs_by_beer['spacy_similarity'] = spacy_by_beer
all_recs_by_beer['Recommendation_Score_Spacy'] = scale(spacy_by_beer) + scale(all_recs_by_beer['overall_sent_score'])

# sort by recommendation score calculated using Spacy instead of BoW cosine similarity
add_spacy = all_recs_by_beer.sort_values(by='Recommendation_Score_Spacy', ascending=False)
result_F = add_spacy[['similarity_score','spacy_similarity','user_rating','overall_sent_score','Recommendation_Score','Recommendation_Score_Spacy']]
result_F

,similarity_score,spacy_similarity,user_rating,overall_sent_score,Recommendation_Score,Recommendation_Score_Spacy
product_name_x,,,,,,
Headroom,0.000000,0.689513,4.5168,0.750100,2.642698,4.848193
Pirate Bomb!,0.010105,0.688888,4.4764,0.649175,3.313941,3.976546
Fou' Foune,0.020253,0.712967,4.5068,0.496250,3.556548,3.754378
Abraxas,0.019852,0.742988,4.4980,0.283637,1.718113,3.293760
Zenne Y Frontera,0.031206,0.711061,4.7068,0.388415,4.297057,2.768732
...,...,...,...,...,...,...
Assassin,0.005486,0.658370,4.5012,0.000000,-2.808800,-2.792974
Resolute - Coconut,0.005877,0.635304,4.6032,0.114967,-1.788539,-2.843835
Pliny The Elder,0.008355,0.638247,4.5912,0.075800,-1.744537,-3.042231


In [32]:
result_F[:3]

,similarity_score,spacy_similarity,user_rating,overall_sent_score,Recommendation_Score,Recommendation_Score_Spacy
product_name_x,,,,,,
Headroom,0.000000,0.689513,4.5168,0.750100,2.642698,4.848193
Pirate Bomb!,0.010105,0.688888,4.4764,0.649175,3.313941,3.976546
Fou' Foune,0.020253,0.712967,4.5068,0.496250,3.556548,3.754378


In [33]:
result_F.loc['Leche Borracho'] # no longer in top 3

similarity_score              0.038135
spacy_similarity              0.728832
user_rating                   4.475600
overall_sent_score            0.179663
Recommendation_Score          3.590068
Recommendation_Score_Spacy    1.802690
Name: Leche Borracho, dtype: float64

Using spaCy similarity in place of BoW cosine similarity, we found that two of our original top three recommendations were replaced with other beer brands. This is likely because bag of words is more particular about finding the exact attribute whereas spaCy might assume attributes that are often mentioned together are similar when they are in fact distinct - like spicy and aroma. For instance, one of our original top three beers, Leche Borracho, included many reviews that mentioned all three attributes. However, phrases like "spicy aroma" were mentioned, the occurrence of which vanilla cosine similarity would have awarded a much higher similarity score compared to that from spaCy. As a result, the similarity score contribution (after scaling) to the overall recommendation score for Leche Borracho using spaCy is reduced compared to that from BoW because these two attributes are considered similar and not distinct.

# Task G
How would our recommendations differ if we ignored similarity and feature sentiment scores? Below backs up our conclusion with analysis.

In [36]:
result_F.sort_values(by='user_rating', ascending = False)[:3]

,similarity_score,spacy_similarity,user_rating,overall_sent_score,Recommendation_Score,Recommendation_Score_Spacy
product_name_x,,,,,,
Chemtrailmix,0.013779,0.684733,4.754286,0.01032,-1.478809,-1.549475
It Was All A Dream,0.014934,0.669450,4.747200,0.20800,0.347813,-0.566802
SR-71,0.001513,0.605507,4.746400,0.00000,-3.404597,-5.113390


If we just return the top three rated beers in the dataset, we see the three most highly rated beers among users, irrespective of specific attributes. The point of a recommendation system is to make meaniningful suggestions to a user based on their preferences; returning the three most highly rated overall defeats that purpose. Had we not analyzed the beers using cosine similarity or spacy similarity, and simply returned the most popular beers, we would have simply given the user a list of three beers that other people found good, for reasons wholly unrelated to their preferences. That would lead the user to likely not use the recommendation system again. It's essentially a coin toss as to whether they will like the beer given to them and that is not sufficient. A good recommendation system should take the attributes from the user as inputs, then find the beers most closely associated with those attributes, so the user has the opportunity to try a beer that they are likely to enjoy.